## Load Data Set

In [1]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical


print(tf.__version__)


root_path = './'

models_path = os.path.join(root_path, 'saved_models_mnist_v7')
if not os.path.exists(models_path):
    os.mkdir(models_path)


mnist = keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = (x_train / 127.5) - 1, (x_test / 127.5) - 1


BATCH_SIZE = 256

Using TensorFlow backend.


1.13.1


In [2]:
print(x_train.shape, y_train.shape)
x_train = np.expand_dims(x_train, -1)
y_train = to_categorical(y_train)
print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)
(60000, 28, 28, 1) (60000, 10)


In [3]:
def custom_loader():

  trainset_size = x_train.shape[0]

  p = 0
  while True:

    idx_from = (p * BATCH_SIZE) % trainset_size

    idx_to = idx_from + BATCH_SIZE

    batch_x = x_train[idx_from: idx_to]
    batch_y = y_train[idx_from: idx_to]
    
    p += 1
  
    yield batch_x, batch_y

In [4]:
custom_gen = custom_loader()
for i in range(3):
  batch_x, batch_y = next(custom_gen)
  print(batch_x.shape, batch_y.shape)

(256, 28, 28, 1) (256, 10)
(256, 28, 28, 1) (256, 10)
(256, 28, 28, 1) (256, 10)


## Model Design

In [ ]:
enc_len = 0
age_len = 10
img_shape = (28, 28, 1)
width, height, depth = (28, 28, 1)
img_len = np.prod(img_shape)
latent_dim = enc_len + age_len + img_len
noise_len = 50  # 32 x 32 x 3
input_dim = enc_len + age_len + noise_len
cond_len = enc_len + age_len


def build_discriminator():
    conv = keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=16,
            kernel_size=(3, 3),
            strides=2,
            input_shape=img_shape
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        keras.layers.BatchNormalization(),

        # conv block 2
        keras.layers.Conv2D(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        keras.layers.BatchNormalization(),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=3,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
    ])
    
    print("D M1:")
    conv.summary()
    
    model = keras.Sequential([
        # output
        keras.layers.Dense(1, input_shape=(age_len+12,)),
        keras.layers.Activation(tf.nn.sigmoid),
    ])
    
    print("D M2:")
    model.summary()
    
    # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # image
    z = keras.layers.Input(shape=img_shape)
    
    # convolution
    zout = conv(z)
    
    # flatten image
    z_flat = keras.layers.Flatten()(zout)
    
    # concatenation
    inputs = keras.layers.concatenate([c2, z_flat])
    
    # real or fake
    outputs = model(inputs)
    
    return keras.models.Model([c2, z], outputs)


def build_generator():
    
    conv = keras.Sequential([
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=16,
            kernel_size=(3, 3),
            strides=1,
            input_shape=(11, 11, 1)
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(2, 2),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),

        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=1,
            kernel_size=(1, 1),
            strides=1
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])
    
    print("G M2:")
    conv.summary()
    
    model = keras.Sequential([
        # dense 1
        keras.layers.Dense(121, input_shape=(input_dim,)),
        keras.layers.Activation(tf.nn.relu),
        
        # reshape 1d to 3d
        keras.layers.Reshape((11, 11, 1))
    ])
    
    print("G M1:")
    model.summary()
    
    # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
    c2 = keras.layers.Input(shape=(age_len,))
    
    # noise
    x = keras.layers.Input(shape=(noise_len,))

    # concatenation
    inputs = keras.layers.concatenate([c2, x])
    
    # flat dense output
    out_1 = model(inputs)
    
    # transpose conv output
    outputs = conv(out_1)
    
    return keras.models.Model([c2, x], outputs)


discriminator = build_discriminator()
generator = build_generator()

Instructions for updating:
Colocations handled automatically by placer.
D M1:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 13, 13, 16)        160       
_________________________________________________________________
activation_1 (Activation)    (None, 13, 13, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 13, 13, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          4640      
_________________________________________________________________
activation_2 (Activation)    (None, 6, 6, 32)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 6, 6, 32)          128       
________________________________________________________________

In [ ]:
generator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 60)           0           input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
sequential_4 (Sequential)       (None, 11, 11, 1)    7381        concatenate_2[0][0]              
__________

In [ ]:
discriminator.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 2, 2, 3)      5859        input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 12)           0           sequential_1[1][0]               
__________________________________________________________________________________________________
concatenat

## Generative Adversarial Network

In [ ]:
GLR = 0.02  # generator
DLR = 0.02  # discriminator


discriminator.compile(
    optimizer=keras.optimizers.Adam(DLR, 0.5),
    loss=keras.losses.mse,
    metrics=['accuracy']
)


# condition
# c1 = keras.layers.Input(shape=(enc_len,))
c2 = keras.layers.Input(shape=(age_len,))

# noise
x = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
z = generator([c2, x])
out = discriminator([c2, z])

# GAN
gan = keras.models.Model(inputs=[c2, x], outputs=out)

gan.compile(
    optimizer=keras.optimizers.Adam(GLR , 0.5),
    loss=keras.losses.mse,
    metrics=['accuracy'])

In [ ]:
gan.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 28, 28, 1)    16662       input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            5882        input_5[0][0]                    
          

## Visualization Method

In [ ]:
# from google.colab import drive
import os


# drive.mount('/content/gdrive', force_remount=True)

root_path = './'
tgt_pth = os.path.join(root_path, 'visualize_mnist-v19')

if not os.path.exists(tgt_pth):
  os.mkdir(tgt_pth)

In [ ]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(4, 4, figsize=(20, 18))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_real[r_real * 4 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )
            else:
                cell.imshow(
                    np.squeeze(
                        0.5 * z_fake[r_fake * 4 + col] + 0.5,
                        axis=-1
                    ),
                    cmap='gray'
                )

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    plt.tight_layout()

    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

## Load Batch

In [ ]:
def load_noise():
    
    y_true = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_true = y_true.eval()

        while True:
            batch_x, batch_y = next(custom_gen)

            sz = batch_x.shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            c2 = tf.cast(batch_y, tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            
            yield c2, x, y_true


def load_batch():
    
    y_fake = tf.zeros((BATCH_SIZE,))
    y_true = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_fake = y_fake.eval()
        y_true = y_true.eval()

        while True:
            batch_x, batch_y = next(custom_gen)

            sz = batch_x.shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            c2 = tf.cast(batch_y, tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            z_fake = generator.predict([c2, x])

            # real data
            c2 = tf.cast(batch_y, tf.float32).eval()
            z_real = batch_x
                        
            yield c2, x, z_fake, y_fake, z_real, y_true

## Train Model

In [ ]:
EPOCHS = 2000
STEPS = 1  # 60000 // BATCH_SIZE


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


disc_itr = load_batch()
gen_itr = load_noise()


# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS):
        
        c2, x, z_fake, y_fake, z_real, y_real = next(disc_itr)
    
        # train
        loss_2, acc_2 = discriminator.train_on_batch([c2, z_real], y_real)
        loss_1, acc_1 = discriminator.train_on_batch([c2, z_fake], y_fake)

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

      c2, x, y_true = next(gen_itr)

      # train
      loss_1, acc_1 = gan.train_on_batch([c2, x], y_true)

      loss.append(loss_1)
      acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("E: {}, D:[ACC: %{:.2f}, LOSS: {:.2f}], G:[ACC: %{:.2f}, LOSS: {:.2f}]".format(
          e,
          train_acc_d[-1] * 100,
          train_loss_d[-1] * 100,
          train_acc_g[-1] * 100,
          train_loss_g[-1] * 100
      ))

    if e % 100 == 0:
        ## visualize results
        c2, x, z_fake, y_fake, z_real, y_real = next(disc_itr)
        visualizeGAN(e, z_real, z_fake)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


E: 0, D:[ACC: %52.54, LOSS: 34.31], G:[ACC: %43.36, LOSS: 30.13]
E: 1, D:[ACC: %48.63, LOSS: 34.91], G:[ACC: %20.70, LOSS: 54.06]
E: 2, D:[ACC: %65.62, LOSS: 21.87], G:[ACC: %9.38, LOSS: 64.69]
E: 3, D:[ACC: %67.97, LOSS: 24.43], G:[ACC: %34.77, LOSS: 43.47]
E: 4, D:[ACC: %57.62, LOSS: 28.74], G:[ACC: %39.06, LOSS: 38.37]
E: 5, D:[ACC: %46.09, LOSS: 37.74], G:[ACC: %59.77, LOSS: 25.75]
E: 6, D:[ACC: %68.16, LOSS: 22.00], G:[ACC: %21.88, LOSS: 55.83]
E: 7, D:[ACC: %79.30, LOSS: 14.05], G:[ACC: %9.77, LOSS: 72.73]
E: 8, D:[ACC: %75.98, LOSS: 15.61], G:[ACC: %23.83, LOSS: 61.54]
E: 9, D:[ACC: %67.38, LOSS: 21.68], G:[ACC: %16.02, LOSS: 60.22]
E: 10, D:[ACC: %53.32, LOSS: 31.99], G:[ACC: %21.09, LOSS: 56.72]
E: 11, D:[ACC: %64.84, LOSS: 22.86], G:[ACC: %11.33, LOSS: 66.84]
E: 12, D:[ACC: %63.09, LOSS: 22.15], G:[ACC: %12.11, LOSS: 58.82]
E: 13, D:[ACC: %45.31, LOSS: 30.14], G:[ACC: %17.19, LOSS: 54.05]
E: 14, D:[ACC: %88.09, LOSS: 10.88], G:[ACC: %3.52, LOSS: 70.82]
E: 15, D:[ACC: %47.85, 

E: 126, D:[ACC: %59.96, LOSS: 23.79], G:[ACC: %35.16, LOSS: 33.51]
E: 127, D:[ACC: %81.25, LOSS: 14.48], G:[ACC: %53.12, LOSS: 28.02]
E: 128, D:[ACC: %68.36, LOSS: 20.01], G:[ACC: %57.03, LOSS: 25.04]
E: 129, D:[ACC: %54.10, LOSS: 26.59], G:[ACC: %50.78, LOSS: 26.45]
E: 130, D:[ACC: %56.45, LOSS: 24.72], G:[ACC: %43.75, LOSS: 30.22]
E: 131, D:[ACC: %61.52, LOSS: 24.17], G:[ACC: %52.73, LOSS: 25.95]
E: 132, D:[ACC: %82.03, LOSS: 14.33], G:[ACC: %66.41, LOSS: 20.77]
E: 133, D:[ACC: %77.93, LOSS: 15.51], G:[ACC: %48.05, LOSS: 30.95]
E: 134, D:[ACC: %43.16, LOSS: 35.07], G:[ACC: %21.88, LOSS: 46.69]
E: 135, D:[ACC: %81.84, LOSS: 13.84], G:[ACC: %35.16, LOSS: 36.94]
E: 136, D:[ACC: %78.52, LOSS: 15.03], G:[ACC: %54.30, LOSS: 27.32]
E: 137, D:[ACC: %73.24, LOSS: 18.18], G:[ACC: %27.34, LOSS: 41.45]
E: 138, D:[ACC: %39.84, LOSS: 35.54], G:[ACC: %3.91, LOSS: 67.82]
E: 139, D:[ACC: %23.63, LOSS: 44.19], G:[ACC: %23.05, LOSS: 38.59]
E: 140, D:[ACC: %47.07, LOSS: 28.97], G:[ACC: %25.39, LOSS: 37.

E: 250, D:[ACC: %35.35, LOSS: 32.16], G:[ACC: %17.19, LOSS: 42.48]
E: 251, D:[ACC: %48.63, LOSS: 27.93], G:[ACC: %21.48, LOSS: 35.15]
E: 252, D:[ACC: %66.60, LOSS: 22.75], G:[ACC: %36.33, LOSS: 29.79]
E: 253, D:[ACC: %72.07, LOSS: 18.68], G:[ACC: %50.78, LOSS: 27.37]
E: 254, D:[ACC: %66.60, LOSS: 19.67], G:[ACC: %47.66, LOSS: 29.28]
E: 255, D:[ACC: %63.09, LOSS: 22.69], G:[ACC: %15.23, LOSS: 38.97]
E: 256, D:[ACC: %51.37, LOSS: 28.06], G:[ACC: %8.98, LOSS: 41.71]
E: 257, D:[ACC: %58.01, LOSS: 24.36], G:[ACC: %10.16, LOSS: 42.36]
E: 258, D:[ACC: %45.12, LOSS: 28.56], G:[ACC: %11.72, LOSS: 42.18]
E: 259, D:[ACC: %64.65, LOSS: 22.59], G:[ACC: %11.72, LOSS: 46.85]
E: 260, D:[ACC: %59.57, LOSS: 24.46], G:[ACC: %10.94, LOSS: 46.15]
E: 261, D:[ACC: %52.54, LOSS: 26.10], G:[ACC: %15.62, LOSS: 45.38]
E: 262, D:[ACC: %45.12, LOSS: 29.47], G:[ACC: %21.48, LOSS: 38.56]
E: 263, D:[ACC: %51.37, LOSS: 26.14], G:[ACC: %19.53, LOSS: 41.34]
E: 264, D:[ACC: %55.27, LOSS: 26.31], G:[ACC: %17.58, LOSS: 40.

E: 374, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 375, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 376, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 377, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 378, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 379, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 380, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 381, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 382, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 383, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 384, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 385, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 386, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 387, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 388, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100

E: 497, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 498, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 499, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 500, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 501, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 502, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 503, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 504, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 505, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 506, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 507, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 508, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 509, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]
E: 510, D:[ACC: %50.00, LOSS: 50.00], G:[ACC: %0.00, LOSS: 100.00]


## Plot Loss

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

## Plot Accuracy

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();

In [ ]:
generator.save('./mnist-gen.h5')

In [ ]:
discriminator.save('./mnist-disc.h5')

In [ ]:
gan.save('./mnist-gan.h5')